# pretrained gene embedding, without expresssion
## 1. scbert 
1. read the config file(.toml)
2. init the LoadScbert object
3. run the get_embedding() func.
4. evaluate the embedding

In [3]:
from biollm.model.grn import GeneEmbedding
from biollm.evaluate.bm_metrices_grn import evaluate, get_genesets_dict
from collections import defaultdict
import pandas as pd


file_path = '../../case/data/gene_setinfo.pkl'
gene_pw_dict = get_genesets_dict(file_path)
def evalue_gene_emb(emb, gene_pw_dict):
    grn_embed = GeneEmbedding(emb)
    result = defaultdict(list)
    for i in range(1, 9):
        threshold = 0.1 * i
        g = grn_embed.generate_network(threshold=threshold)
        res = evaluate(g, gene_pw_dict)
        result['threshold'].append(threshold)
        result['Modularity'].append(res['Modularity'])
        result['Biological_index'].append(res['Biological Index'])
    return pd.DataFrame(result)

In [5]:
from biollm.utils.utils import load_config
import numpy as np
from biollm.base.load_scbert import LoadScbert
import torch
import pickle as pkl
import os


config_file = './configs/scbert_gene_emb.toml'
configs = load_config(config_file)

obj = LoadScbert(configs)
print(obj.args)

gene_ids = list(obj.get_gene2idx().values())
gene_ids = np.array(gene_ids)
gene_ids = torch.tensor(gene_ids, dtype=torch.long).to(configs.device)
obj.model = obj.model.to(configs.device)
emb = obj.get_embedding(obj.args.emb_type, gene_ids=gene_ids)
print('embedding shape:', emb.shape)
if not os.path.exists(configs.output_dir):
    os.makedirs(configs.output_dir, exist_ok=True)
with open(obj.args.output_dir + f'/scbert_{obj.args.emb_type}_emb.pk', 'wb') as w:
    res = {'gene_names': list(obj.get_gene2idx().keys()), 'gene_emb': emb}
    pkl.dump(res, w)

2024-07-29 16:55:53-load_llm[line-50]-INFO: Loading params token_emb.weight with shape torch.Size([7, 200])
2024-07-29 16:55:53-load_llm[line-50]-INFO: Loading params pos_emb.emb.weight with shape torch.Size([16907, 200])
2024-07-29 16:55:53-load_llm[line-50]-INFO: Loading params performer.calls_since_last_redraw with shape torch.Size([])
2024-07-29 16:55:53-load_llm[line-50]-INFO: Loading params performer.net.layers.0.0.norm.weight with shape torch.Size([200])
2024-07-29 16:55:53-load_llm[line-50]-INFO: Loading params performer.net.layers.0.0.norm.bias with shape torch.Size([200])
2024-07-29 16:55:53-load_llm[line-50]-INFO: Loading params performer.net.layers.0.0.fn.fast_attention.projection_matrix with shape torch.Size([266, 64])
2024-07-29 16:55:53-load_llm[line-50]-INFO: Loading params performer.net.layers.0.0.fn.to_q.weight with shape torch.Size([640, 200])
2024-07-29 16:55:53-load_llm[line-50]-INFO: Loading params performer.net.layers.0.0.fn.to_k.weight with shape torch.Size([640

Munch({'model_used': 'scbert', 'emb_type': 'gene', 'model_file': '/home/share/huadjyin/home/s_qiuping1/workspace/omics_model/scBERT/ckpt/panglao_pretrain.pth', 'vocab_file': '/home/share/huadjyin/home/s_qiuping1/workspace/omics_model/data/train_data/gene2vec/gene_vocab.json', 'model_type': 'scbert', 'output_dir': '../../case/result/zero-shot/', 'device': 'cuda:2', 'bin_num': 7, 'embsize': 200, 'distributed': False, 'batch_size': 8, 'max_seq_len': 16907, 'use_g2v': True, 'g2v_file': '/home/share/huadjyin/home/s_qiuping1/workspace/omics_model/data/train_data/gene2vec/gene2vec_16906.npy', 'weight_bias_track': False, 'project_name': 'biollm', 'exp_name': 'scbert_gene_emb_gpu'})
gpu used:  0
embedding shape: (16906, 200)


In [17]:
(torch.cuda.memory_cached('cuda:2') + torch.cuda.memory_reserved('cuda:2')) / (1024**2)

152.0

In [15]:
torch.cuda.mem_get_info()

(22834380800, 42314694656)

## 2. scgpt

In [6]:
from biollm.utils.utils import load_config
import numpy as np
from biollm.base.load_scgpt import LoadScgpt
import torch
import pickle as pkl
import os


config_file = './configs/scgpt_gene_emb.toml'
configs = load_config(config_file)

obj = LoadScgpt(configs)
print(obj.args)

gene_ids = list(obj.get_gene2idx().values())
gene_ids = np.array(gene_ids)
gene_ids = torch.tensor(gene_ids, dtype=torch.long).to(configs.device)
obj.model = obj.model.to(configs.device)
emb = obj.get_embedding(obj.args.emb_type, gene_ids=gene_ids)
print('embedding shape:', emb.shape)
if not os.path.exists(configs.output_dir):
    os.makedirs(configs.output_dir, exist_ok=True)
with open(obj.args.output_dir + f'/scgpt_{obj.args.emb_type}_emb.pk', 'wb') as w:
    res = {'gene_names': list(obj.get_gene2idx().keys()), 'gene_emb': emb}
    pkl.dump(res, w)

{'ntoken': 60697, 'd_model': 512, 'nhead': 8, 'd_hid': 512, 'nlayers': 12, 'nlayers_cls': 3, 'n_cls': 1, 'dropout': 0.2, 'pad_token': '<pad>', 'do_mvc': False, 'do_dab': False, 'use_batch_labels': False, 'num_batch_labels': None, 'domain_spec_batchnorm': False, 'input_emb_style': 'continuous', 'cell_emb_style': 'cls', 'mvc_decoder_style': 'inner product', 'ecs_threshold': 0.3, 'explicit_zero_prob': False, 'fast_transformer_backend': 'flash', 'pre_norm': False, 'vocab': GeneVocab(), 'pad_value': -2, 'n_input_bins': 51, 'use_fast_transformer': True}


2024-07-29 16:55:59-load_llm[line-50]-INFO: Loading params encoder.embedding.weight with shape torch.Size([60697, 512])
2024-07-29 16:55:59-load_llm[line-50]-INFO: Loading params encoder.enc_norm.weight with shape torch.Size([512])
2024-07-29 16:55:59-load_llm[line-50]-INFO: Loading params encoder.enc_norm.bias with shape torch.Size([512])
2024-07-29 16:55:59-load_llm[line-50]-INFO: Loading params value_encoder.linear1.weight with shape torch.Size([512, 1])
2024-07-29 16:55:59-load_llm[line-50]-INFO: Loading params value_encoder.linear1.bias with shape torch.Size([512])
2024-07-29 16:55:59-load_llm[line-50]-INFO: Loading params value_encoder.linear2.weight with shape torch.Size([512, 512])
2024-07-29 16:55:59-load_llm[line-50]-INFO: Loading params value_encoder.linear2.bias with shape torch.Size([512])
2024-07-29 16:55:59-load_llm[line-50]-INFO: Loading params value_encoder.norm.weight with shape torch.Size([512])
2024-07-29 16:55:59-load_llm[line-50]-INFO: Loading params value_encoder

Munch({'model_used': 'scgpt', 'emb_type': 'gene', 'model_file': '../../case/models/scgpt/best_model.pt', 'model_param_file': '../../case/models/scgpt/args.json', 'vocab_file': '../../case/models/scgpt/vocab.json', 'output_dir': '../../case/result/zero-shot/', 'pad_value': -2, 'mask_ratio': 0, 'device': 'cuda', 'CLS': False, 'ADV': False, 'CCE': False, 'MVC': False, 'ECS': False, 'weight_bias_track': False, 'project_name': 'biollm', 'exp_name': 'scgpt_gene_emb_gpu', 'data_source': '/scratch/ssd004/datasets/cellxgene/scb_strict/human', 'save_dir': '/scratch/ssd004/datasets/cellxgene/save/cellxgene_census_human-May23-08-36-2023', 'load_model': None, 'n_hvg': None, 'valid_size_or_ratio': 0.003, 'dist_backend': 'nccl', 'grad_accu_steps': 1, 'pad_token': '<pad>', 'input_style': 'binned', 'input_emb_style': 'continuous', 'n_bins': 51, 'max_seq_len': 1200, 'training_tasks': 'both', 'dist_url': 'tcp://gpu188.cluster.local:53833', 'trunc_by_sample': True, 'vocab_path': '/scratch/ssd004/datasets/

## 3. geneformer

In [7]:
from biollm.utils.utils import load_config
import numpy as np
from biollm.base.load_geneformer import LoadGeneformer
import torch
import pickle as pkl
import os


config_file = './configs/geneformer_gene_emb.toml'
configs = load_config(config_file)

obj = LoadGeneformer(configs)
print(obj.args)

gene_ids = list(obj.get_gene2idx().values())
gene_ids = np.array(gene_ids)
gene_ids = torch.tensor(gene_ids, dtype=torch.long).to(configs.device)
obj.model = obj.model.to(configs.device)
emb = obj.get_embedding(obj.args.emb_type, gene_ids=gene_ids)
print('embedding shape:', emb.shape)
if not os.path.exists(configs.output_dir):
    os.makedirs(configs.output_dir, exist_ok=True)
with open(obj.args.output_dir + f'/geneformer_{obj.args.emb_type}_emb.pk', 'wb') as w:
    res = {'gene_names': list(obj.get_gene2idx().keys()), 'gene_emb': emb}
    pkl.dump(res, w)

Munch({'model_used': 'geneformer', 'model_type': 'Pretrained', 'model_file': '/home/share/huadjyin/home/s_huluni/project/bio_model_240614/models/geneformer', 'vocab_file': '/home/share/huadjyin/home/s_huluni/project/bio_model_240614/models/geneformer/gene_vocab.json', 'n_bins': 100, 'input_file': '/home/share/huadjyin/home/s_huluni/project/bio_model/dataset/Zheng68K.h5ad', 'output_dir': '../../case/result/zero-shot/', 'device': 'cuda', 'emb_type': 'gene', 'encoder_dims': 768, 'encoder_heads': 12, 'encoder_depth': 12, 'max_seq_len': 19264, 'use_g2v': False, 'cell_type_key': 'celltype', 'weight_bias_track': False, 'project_name': 'biollm', 'exp_name': 'geneformer_gene_emb_gpu'})
embedding shape: torch.Size([25426, 256])


In [26]:
obj.get_gene2idx()['ENSGR0000214717']

25424

In [20]:
gene_ids

tensor([25424, 25423, 25421,  ...,  1773, 13918, 20751], device='cuda:0')

In [24]:
obj.model.bert.embeddings.word_embeddings(gene_ids).shape

torch.Size([25426, 256])

## 4. scfoundation

In [8]:
from biollm.utils.utils import load_config
import numpy as np
from biollm.base.load_scfoundation import LoadScfoundation
import torch
import pickle as pkl
import os


config_file = './configs/scfoundation_gene_emb.toml'
configs = load_config(config_file)

obj = LoadScfoundation(configs)
print(obj.args)

gene_ids = list(obj.get_gene2idx().values())
gene_ids = np.array(gene_ids)
gene_ids = torch.tensor(gene_ids, dtype=torch.long).to(configs.device)
obj.model = obj.model.to(configs.device)
emb = obj.get_embedding(configs.emb_type, gene_ids=gene_ids)
print('embedding shape:', emb.shape)
if not os.path.exists(configs.output_dir):
    os.makedirs(configs.output_dir, exist_ok=True)
with open(obj.args.output_dir + f'/scfoundation_{obj.args.emb_type}_emb.pk', 'wb') as w:
    res = {'gene_names': list(obj.get_gene2idx().keys()), 'gene_emb': emb}
    pkl.dump(res, w)

{'mask_gene_name': False, 'gene_num': 19266, 'seq_len': 19266, 'encoder': {'hidden_dim': 768, 'depth': 12, 'heads': 12, 'dim_head': 64, 'seq_len': 19266, 'module_type': 'transformer', 'norm_first': False}, 'decoder': {'hidden_dim': 512, 'depth': 6, 'heads': 8, 'dim_head': 64, 'module_type': 'performer', 'seq_len': 19266, 'norm_first': False}, 'n_class': 104, 'pad_token_id': 103, 'mask_token_id': 102, 'bin_num': 100, 'bin_alpha': 1.0, 'rawcount': True, 'model': 'mae_autobin', 'test_valid_train_idx_dict': '/nfs_beijing/minsheng/data/os10000w-new/global_shuffle/meta.csv.train_set_idx_dict.pt', 'valid_data_path': '/nfs_beijing/minsheng/data/valid_count_10w.npz', 'num_tokens': 13, 'train_data_path': None, 'isPanA': False, 'isPlanA1': False, 'max_files_to_load': 5, 'bin_type': 'auto_bin', 'value_mask_prob': 0.3, 'zero_mask_prob': 0.03, 'replace_prob': 0.8, 'random_token_prob': 0.1, 'mask_ignore_token_ids': [0], 'decoder_add_zero': True, 'mae_encoder_max_seq_len': 15000, 'isPlanA': False, 'ma

2024-07-29 16:56:14-load_scfoundation[line-70]-INFO: start to get gene embedding!
2024-07-29 16:56:14-load_scfoundation[line-75]-INFO: start to get gene embedding!


Munch({'model_used': 'scfoundation', 'model_file': '../../case/models/scfoundation/models.ckpt', 'vocab_file': '../../case/models/scfoundation/gene_vocab.json', 'key': 'cell', 'emb_type': 'gene', 'input_type': 'singlecell', 'tgthighres': 't4', 'pool_type': 'all', 'batch_size': 64, 'n_bins': 100, 'input_file': '/home/share/huadjyin/home/s_qiuping1/workspace/omics_model/downstream_tasks/test/grn/data/Immune_ALL_human.h5ad', 'output_dir': '../../case/result/zero-shot/', 'device': 'cuda', 'quantile_cutoff': 0, 'encoder_dims': 128, 'encoder_heads': 12, 'encoder_depth': 12, 'data_is_raw': False, 'max_seq_len': 19264, 'use_g2v': False, 'hvg_number': 3000, 'weight_bias_track': False, 'project_name': 'biollm', 'exp_name': 'scf_gene_emb_gpu'})
embedding shape: (19264, 768)


In [6]:
emb = emb.detach().cpu().numpy()
gene_names = list(obj.get_gene2idx().keys())
genes_embedding = {gene_names[i]: emb[i] for i in range(len(gene_ids))}
df = evalue_gene_emb(genes_embedding, gene_pw_dict)
df

100%|██████████| 185624528/185624528 [03:34<00:00, 865913.02it/s]


KeyboardInterrupt: 